# 3. Operaciones con RDDs

### Lección 7: RDD y DataFrames.

Todas las aplicaciones en Spark poseen un manejador central de programa (Driver) y varios ejecutores que se crean a lo largo del clúster, estas son las computadoras que realizarán las tareas en paralelo y finalmente devolverán los valores al driver, la aplicación central.

Para fines de este curso, debido a que se usa un modelo stand-alone, solo se contará con un driver y un ejecutor, ambos alojados en la misma computadora.

### RDD

Para poder realizar estas tareas, Spark posee desde su versión 1.0 los RDD (Resilient Distributed Dataset), los cuales son tolerantes a fallos y pueden ser distribuidos a lo largo de los nodos del clúster.

Los RDD pueden ser creados al cargar datos de manera distribuida, como es desde un HDFS, Cassanda, Hbase o cualquier sistema de datos soportado por Hadoop, pero también por colecciones de datos de Scala o Python, además de poder ser leídos desde archivos en el sistema local.

En visión general, un RDD puede ser visto como un set de datos los cuales soportan solo dos tipos de operaciones: transformaciones y acciones.

Las transformaciones permiten crear un nuevo RDD a partir de uno previamente existente, mientras que las acciones retornan un valor al driver de la aplicación. El núcleo de operación del paradigma de Spark es la ejecución perezosa (Lazy), es decir que las transformaciones solo serán calculadas posterior a una llamada de acción.

Además, los RDD poseen una familiaridad con el paradigma orientado a objetos, lo cual permite que podamos realizar operaciones de bajo nivel a modo. Map, filter y reduce son tres de las operaciones más comunes.

Una de las grandes ventajas que ofrecen los RDD es la compilación segura; por su particularidad de ejecución perezosa, se calcula si se generará un error o no antes de ejecutarse, lo cual permite identificar problemas antes de lanzar la aplicación. El pero que podemos encontrar con los RDD es que no son correctamente tratados por el Garbage collector y cuando las lógicas de operación se hacen complejas, su uso puede resultar poco práctico, aquí entran los DataFrames.

### DataFrames

Esos componentes fueron agregados en la versión 1.3 de Spark y pueden ser invocados con el contexto espacial de Spark SQL. Como indica su nombre, es un módulo especialmente desarrollado para ser ejecutado con instrucciones parecidas al SQL estándar.

De la misma forma, como los RDD, estos pueden ser creados a partir de archivos, tablas tipo Hive, bases de datos externas y RDD o DataFrames existentes.

El primer detalle que salta cuando creamos un DataFrame es que poseen columnas nombradas, lo que a nivel conceptual es como trabajar con un DataFrame de Pandas. Con la excepción que a nivel interno Spark trabaja con Scala, lo cual le asigna a cada columna el tipo de dato Row, un tipo especial de objeto sin tipo definido.

Pero no es todo, los DataFrames implementan un sistema llamado Catalyst, el cual es un motor de optimización de planes de ejecución, parecido al que usan las bases de datos, pero diseñado para la cantidad de datos propia de Spark, aunado a eso, se tiene implementado un optimizador de memoria y consumo de CPU llamado Tungsten, el cual determina cómo se convertirán los planes lógicos creados por Catalyst a un plan físico.

### Fin de la lección 7.

### Lección 8: Transformaciones y acciones.

In [1]:
from pyspark import SparkContext
sc = SparkContext(master="local", appName="TransformacionesYAcciones") #Creando el Inicializador

In [2]:
#El parámetro master = "local" indica dónde estará habitando el contexto, 
#en este caso en la computadora local.

In [3]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)
#Con el metodo .parallelize podremos pasar una serie de datos directamente

pyspark.rdd.RDD

In [4]:
rdd1.collect()

[1, 2, 3]

In [5]:
sc


<SparkContext master=local appName=TransformacionesYAcciones>

In [6]:
!ls

codeExample.py
data.csv
files
Leccion_10_Acciones_de_conteo_sobre_RDDs.ipynb
Leccion_11_Solucion_reto_deportistas.ipynb
Leccion_6_Jupyter_vs_CLI.ipynb
Leccion_8_Transformaciones_y_Acciones.ipynb
Leccion_9_Acciones_de_modificacion_sobre_RDDs.ipynb
modelo_relacional_4ce1ab04-d36f-4c9e-afd0-03e4a7d874b9.jpeg
Seccion_3_Operaciones_sobre_RDD.ipynb
Seccion_4_Data_Frames_y_SQL.ipynb
spark-warehouse
venv


In [7]:
!pwd

/home/cleto/Desktop/Intro_to_Spark


In [8]:
path = "./files/"
equiposOlimpicosRDD = sc.textFile(path + "paises.csv")\
    .map(lambda line : line.split(","))

In [9]:
equiposOlimpicosRDD.take(15)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG'],
 ['5', 'Afghanistan', 'AFG'],
 ['6', 'Akatonbo', 'IRL'],
 ['7', 'Alain IV', 'SUI'],
 ['8', 'Albania', 'ALB'],
 ['9', 'Alcaid', 'POR'],
 ['10', 'Alcyon-6', 'FRA'],
 ['11', 'Alcyon-7', 'FRA'],
 ['12', 'Aldebaran', 'ITA'],
 ['13', 'Aldebaran II', 'ITA'],
 ['14', 'Aletta', 'IRL']]

### Fin de la lección 8.

### Lección 9: Acciones de modificación sobre RDDs

In [10]:
equiposOlimpicosRDD.map(lambda x: (x[2])).distinct().count()

231

En la celda anterior podemos notar el mapping del RDD equiposOlimpicosRDD, la función lambda toma la tupla (de una componente, que representa la columna de siglas) y en conjunto con los métodos ```distinct``` y ```count``` cuentan los valores únicos de la columna respectiva.

In [11]:
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey().mapValues(len).take(10)

[('sigla', 1),
 ('AUT', 11),
 ('MEX', 9),
 ('ARG', 18),
 ('AFG', 1),
 ('IRL', 7),
 ('SUI', 17),
 ('ALB', 1),
 ('POR', 21),
 ('FRA', 155)]

En esta celda, por ejemplo, se hace un agrupamiento por llave (método ```gropByKey```). En la función lambda se establece la tupla ```(x[2],x[1])```, donde ```x[2]``` será la llave de agrupación. El método ```mapValues(len)``` devuelve la longitud de la lista producto del arupamiento (recordemos que el resultado de esta transformación son tuplas de dos componentes).

In [12]:
equiposOlimpicosRDD.map(lambda x: (x[2],x[1])).groupByKey().mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

En esta celda reemplazamos el argumento del método ```mapValues(len)``` por ```mapValues(list)``` . Lo que devuelve una transformación con tuplas de dos componentes, donde la segunda componente es la lista de equipos de cada pais.

In [13]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda l: "ARG" in l )
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

El método ```filter``` en conjunto con la función ```(lambda l: "ARG" in l )```regresa una transformación donde solo aparecen equipos con el argumento ```ARG``` en alguna de las filas. Cabe aclarar que el criterio para usar la función ```collect``` sobre ```take``` es la cantidad de datos expresados en la nueva transformación

In [14]:
equiposOlimpicosRDD.count()

1185

In [15]:
equiposOlimpicosRDD.countApprox(1)

1185

En estas celdas podemos ver dos métodos para saber la cantidad de datos que contiene el RDD; por un lado tenemos ```count()```, que devuelve la cantidad de filas que contiene la estructura. Por el otro lado tenemos ```countApprox```, que requiere un numero natural positivo como argumento, el argumento representa el número de milisec que tiene la función para devolver el conteo.

El problema con ```count()``` recae en la cantidad de datos que posee el RDD, ya que puede demorar segundos o incluso minutos para devolver el resultado (si hablamos de miles de millones de datos). Para evitar esto, podriamos usar ```countApprox```, que nos devuelve un valor aproximado al finalizar el tiempo de tolerancia especificado en el argumento de la función.

### Fin de la lección 9

### Lección 10: Acciones de conteo sobre RDDs

In [16]:
!ls ./files/

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


En el directorio de files podemos apreciar dos archivos (deportista y deportista2.csv), estos archivos son parte de un solo archivo masivo. A continuaciòn asignaremos RDDs a cada una de las partes y posteriormente las uniremos en una sola estructura.

In [17]:
deportistaOlimpicoRDD = sc.textFile(path + "deportista.csv")\
    .map(lambda l : l.split(","))
deportistaOlimpicoRDD2 = sc.textFile(path + "deportista2.csv")\
    .map(lambda l : l.split(","))

In [18]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

El método ```union``` concatena dos RDDs en una sola entidad. Debido a que estas operaciones funcionan como una caja negra en la cual no sabemos si las transformaciones y/o acciones corrieron de manera correcta, serecomienda hacer una acción sobre el RDD resultante que involucre a todos los datos del conjunto de datos (vease la siguiente celda)

In [19]:
deportistaOlimpicoRDD.count()

135572

Una acción como ```count``` nos puede dar alguna pista sobre la integridad de la unión; si al ejecutar esta funciòn retorna algún valor, entonces la integridad es suficiente, de lo contrario tenemos un problema que arreglar.

In [20]:
equiposOlimpicosRDD.top(2)

[['id', 'equipo', 'sigla'], ['999', 'Stella-2', 'NOR']]

In [21]:
deportistaOlimpicoRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [22]:
deportistaOlimpicoRDD.map(lambda l: (l[-1],l[:-1]))\
    .join(equiposOlimpicosRDD.map(lambda x: (x[0],x[2])))\
    .take(6)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN'))]

In [23]:
deportistaOlimpicoRDD.map(lambda l: (l[-1],l[:-1]))\
    .join(equiposOlimpicosRDD.map(lambda x: (x[0],x[2])))\
    .takeSample(False, 6, 45)

[('970', (['60014', 'Kim JaeCheon', '1', '20', '175', '73'], 'KOR')),
 ('514', (['37533', 'Yoshihiro Fujita', '1', '32', '176', '100'], 'JPN')),
 ('246', (['105791', 'Dario ari', '1', '22', '207', '110'], 'CRO')),
 ('259', (['49333', 'Ji Holeek', '1', '27', '181', '78'], 'TCH')),
 ('1096', (['82353', 'John Michael Morton', '1', '29', '178', '66'], 'USA')),
 ('1019', (['83074', 'Hans Mller', '1', '0', '0', '0'], 'SUI'))]

In [24]:
resultado = sc.textFile(path + "resultados.csv") \
    .map(lambda l: l.split(","))

In [25]:
resultadoGanador = resultado.filter(lambda l : 'NA' not in l[1])

In [26]:
resultadoGanador.take(2)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4']]

**Reto de esta clase:** Con los RDDs "resultadoGanador" y "equiposOlimpicosRDD", hacer un cruce y obtener un nuevo RDD

### Fin de la lección 10.

### Lección 11: Solución al reto de deportistas.

In [27]:
deportistaPaises = deportistaOlimpicoRDD\
    .map(lambda l: (l[-1],l[:-1]))\
    .join(equiposOlimpicosRDD.map(lambda x:(x[0],x[2])))

In [28]:
deportistaPaises.join(resultadoGanador).take(6)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold')),
 ('74',
  ((['336', 'Arif Yadulla Abdullayev', '1', '27', '164', '60'], 'AZE'),
   'Gold'))]

### Termina la lección 11

### Lección 12: Operaciones numéricas.

In [29]:
valoresMedallas = {'Gold':7,'Silver':5,'Bronze':4}

In [30]:
paisesMedallas = deportistaPaises.join(resultadoGanador)

In [31]:
paisesMedallas.map(lambda x: (x[1][0][-1],x[1][1])).take(4)

[('AZE', 'Gold'), ('AZE', 'Gold'), ('AZE', 'Gold'), ('AZE', 'Gold')]

In [32]:
paisesMedallas = paisesMedallas\
    .map(lambda x: (x[1][0][-1], valoresMedallas[x[1][1]]))

In [33]:
from operator import add
conclusion = paisesMedallas.reduceByKey((add))\
    .sortBy(lambda x : x[1],ascending=False)

In [34]:
conclusion.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

In [35]:
sc.stop()

### Fin de la lección 12

### Fin de la sección 3.